In [153]:
%reset -sf

In [154]:
# LOAD THIS BEFORE START, run rmdl

#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

# import numpy as np
import scipy
import matplotlib.pyplot as plt

abc = "abcdefghijklmnopqrstuvwxyz"
abc_map = {c:i for i,c in enumerate(abc)}
M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
d4 = [(1,0),(0,1),(-1,0),(0,-1)]
d8 = [
    (-1,-1),(-1,0),(-1,1),
    (0,-1),(0,0),(0,1),
    (1,-1),(1,0),(1,1)]
d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [325]:
with open("/Users/htong/Downloads/input.txt") as f:
    srr = f.read().strip().split("\n")

In [311]:
srr = """
#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#
""".strip().split("\n")

In [326]:
arr = [list(row) for row in srr]

In [327]:
n = len(arr)
m = len(arr[0])

In [328]:
class DisjointSet:
    # github.com/not522/ac-library-python/blob/master/atcoder/dsu.py
    # faster implementation of DSU
    def __init__(self, n: int = 0) -> None:
        if n > 0:  # constant size DSU
            self.parent_or_size = [-1] * n
        else:
            # WARNING: non-negative numeric elements only
            self.parent_or_size = defaultdict(lambda: -1)

    def union(self, a: int, b: int) -> int:
        x = self.find(a)
        y = self.find(b)

        if x == y:
            return x

        if -self.parent_or_size[x] < -self.parent_or_size[y]:
            x, y = y, x

        self.parent_or_size[x] += self.parent_or_size[y]
        self.parent_or_size[y] = x

        return x

    def find(self, a: int) -> int:
        parent = self.parent_or_size[a]
        while parent >= 0:
            if self.parent_or_size[parent] < 0:
                return parent
            self.parent_or_size[a], a, parent = (
                self.parent_or_size[parent],
                self.parent_or_size[parent],
                self.parent_or_size[self.parent_or_size[parent]],
            )
        return a

    def size(self, a: int) -> int:
        return -self.parent_or_size[self.find(a)]

In [329]:
# assume DAG

In [330]:
ds = DisjointSet(n*m)

In [331]:
def parse(x,y):
    return x*m + y

def divide(z):
    return divmod(z, m)

In [332]:
for x in range(n):
    for y in range(m):
        if arr[x][y] == "#":
            continue
        for dx, dy in d4:
            xx = x+dx
            yy = y+dy
            if not (0 <= xx < n and 0 <= yy < m):
                continue
            if arr[x][y] == "." and arr[xx][yy] == ".":
                ds.union(parse(x,y), parse(xx,yy))              

In [333]:
mapping = defaultdict(list)

In [334]:
for x in range(n):
    for y in range(m):
        if arr[x][y] == ">":
            mapping[ds.find(parse(x,y-1))].append((ds.find(parse(x,y+1)), 1 + ds.size(parse(x,y+1))))
            mapping[ds.find(parse(x,y+1))].append((ds.find(parse(x,y-1)), 1 + ds.size(parse(x,y-1))))
        if arr[x][y] == "<":
            mapping[ds.find(parse(x,y-1))].append((ds.find(parse(x,y+1)), 1 + ds.size(parse(x,y+1))))
            mapping[ds.find(parse(x,y+1))].append((ds.find(parse(x,y-1)), 1 + ds.size(parse(x,y-1))))
        if arr[x][y] == "^":
            mapping[ds.find(parse(x+1,y))].append((ds.find(parse(x-1,y)), 1 + ds.size(parse(x-1,y))))
            mapping[ds.find(parse(x-1,y))].append((ds.find(parse(x+1,y)), 1 + ds.size(parse(x+1,y))))
        if arr[x][y] == "v":
            mapping[ds.find(parse(x+1,y))].append((ds.find(parse(x-1,y)), 1 + ds.size(parse(x-1,y))))
            mapping[ds.find(parse(x-1,y))].append((ds.find(parse(x+1,y)), 1 + ds.size(parse(x+1,y))))

In [335]:
startpt = ds.find(parse(0,1))
endpt = ds.find(parse(n-1,m-2))

In [336]:
def dfs(cur, visited):
    maxval = -1000000000
    if cur == endpt:
        return 0
    for nex, cost in mapping[cur]:
        if nex in visited:
            continue
        visited.add(cur)
        val = dfs(nex, visited) + cost
        visited.remove(cur)
        maxval = max(maxval, val)
    return maxval

In [337]:
dfs(ds.find(parse(0,1)), set([startpt])) + ds.size(startpt) - 1

6470

In [324]:
mapping

defaultdict(list,
            {120: [(80, 2), (118, 2)],
             80: [(120, 20), (40, 28), (126, 22)],
             40: [(80, 2), (274, 2)],
             118: [(1, 15), (120, 20), (208, 20)],
             1: [(118, 2)],
             126: [(80, 2), (312, 2)],
             208: [(118, 2), (304, 2)],
             274: [(40, 28), (270, 16), (364, 8)],
             270: [(274, 2), (312, 2)],
             304: [(208, 20), (308, 10), (346, 36)],
             312: [(126, 22), (308, 10), (270, 16), (356, 8)],
             364: [(274, 2), (456, 2)],
             308: [(304, 2), (312, 2)],
             346: [(304, 2), (450, 2)],
             356: [(312, 2), (450, 2)],
             450: [(356, 8), (346, 36), (452, 8)],
             456: [(364, 8), (452, 8), (502, 5)],
             452: [(450, 2), (456, 2)],
             502: [(456, 2)]})